# 目标检测7日打卡营作业二：RCNN系列模型实战

## 实战数据集及框架

- 印刷电路板（PCB）瑕疵数据集：[数据下载链接](http://robotics.pkusz.edu.cn/resources/dataset/)，是一个公共的合成PCB数据集，由北京大学发布，其中包含1386张图像以及6种缺陷（缺失孔，鼠标咬伤，开路，短路，杂散，伪铜），用于检测，分类和配准任务。我们选取了其中适用与检测任务的693张图像，随机选择593张图像作为训练集，100张图像作为验证集。
- PaddleDetection：飞桨推出的PaddleDetection是端到端目标检测开发套件，旨在帮助开发者更快更好地完成检测模型的训练、精度速度优化到部署全流程。该框架中提供了丰富的数据增强、网络组件、损失函数等模块，集成了模型压缩和跨平台高性能部署能力。目前基于PaddleDetection已经完成落地的项目涉及工业质检、遥感图像检测、无人巡检等多个领域。

## 作业描述
基于PaddleDetection中的RCNN系列算法，完成印刷电路板（PCB）瑕疵数据集的训练与评估任务，在验证集上采用IoU=0.5，area=all的mAP作为评价指标，得分=mAP * 100，范围[0,100]，分数越高模型效果越好，及格线是60分。鼓励大家使用GPU训练，CPU训练的结果不纳入排行榜。

提示：
1. 增大训练轮数有助于网络收敛的更好，可提高mAP；
2. 增加FPN、更换Backbone等组件可增强网络的表达能力；
2. 在PaddleDetection中configs中有不同种类的RCNN系列模型，可以寻找RCNN系列性能更强的模型配置文件。（注意：由于数据集无物体分割mask信息，所以不支持Mask-RCNN）

## Baseline基线代码
目前基线模型在GPU下训练6个Epoch(1x)后，mAP(IoU=0.5)达到0.57，CPU下在小数据集上微调训练，mAP(IoU=0.5)达到0.649，所以还有很多优化的空间，转动你的大脑开始debug吧^-^

### 数据准备
首先将印刷电路板（PCB）瑕疵数据集与PaddleDetection代码解压到`~/work/`目录中：

In [ ]:
# 解压数据集
!tar -xf data/data52914/PCB_DATASET.tar -C ~/work/
# 解压PaddleDetection源码
!tar -xf data/data52899/PaddleDetection.tar -C ~/work/

### 环境安装
进行训练前需要安装PaddleDetection所需的依赖包，执行以下命令即可安装：

In [2]:
%cd ~/work/PaddleDetection
! pip install -r requirements.txt
! pip install pycocotools

## 修改配置文件
PaddleDetection中提供的配置文件是在8卡GPU环境下的配置，我们需要调整配置文件：
包括最大训练轮数(max_iters)，类别数(num_classes)，学习率(LearningRate)相关参数，数据Reader中TrainReader与EvalReader数据集路径等参数。

**注意：如在CPU下完成本作业，数据集将缩减到很小，训练集50张，测试集10张，但不推荐使用CPU训练**，GPU下请选择`train.json`和`val.json`标注文件，CPU下请选择`train_cpu.json`和`val_cpu.json`标注文件。

## GPU下配置
提示：
- 为保证模型正常训练不出Nan，学习率要根据GPU卡数，batch size变换而做线性变换，比如这里我们将GPU卡数8->1，所以base_lr除以8即可；
- 训练轮数与Epoch转换关系：根据训练集数量与总batch_size大小计算epoch数，然后将epoch数换算得到训练总轮数max_iters。milestones（学习率变化界限）也是同理。配置文件中batch_size=2，训练集数量为593，训练6个Epoch，在单卡GPU上训练，max_iters=593x6=3558。同理计算milestones为: [2372, 3261]
- RCNN系列模型的类别数num_classes需要加上背景background，所以num_classes=6+1=7

### faster_rcnn_r34_1x.yml配置文件修改适配：
- max_iters: 3558
- num_classes: 7
- base_lr: 0.00025
- milestones: [2372, 3261]

### 数据Reader修改适配：
TrainReader:
- image_dir: images
- anno_path: Annotations/train.json
- dataset_dir: /home/aistudio/work/PCB_DATASET

EvalReader:
- image_dir: images
- anno_path: Annotations/val.json
- dataset_dir: /home/aistudio/work/PCB_DATASET

TestReader:
- anno_path: /home/aistudio/work/PCB_DATASET/Annotations/val.json

## CPU下配置
max_iters=50x6=300，同理计算milestones为: [200, 250]

### faster_rcnn_r34_1x.yml配置文件修改适配：
- max_iters: 300
- num_classes: 7
- base_lr: 0.00025
- milestones: [200, 250]

### 数据Reader修改适配：
**选择`train_cpu.json`和`val_cpu.json`标注文件**
TrainReader:
- image_dir: images
- anno_path: Annotations/train_cpu.json
- dataset_dir: /home/aistudio/work/PCB_DATASET

EvalReader:
- image_dir: images
- anno_path: Annotations/val_cpu.json
- dataset_dir: /home/aistudio/work/PCB_DATASET

TestReader:
- anno_path: /home/aistudio/work/PCB_DATASET/Annotations/val_cpu.json

### 配置文件参考
加FPN组件的完整的配置文件，请在[https://github.com/PaddlePaddle/PaddleDetection/tree/release/0.4/configs/pcb](https://github.com/PaddlePaddle/PaddleDetection/tree/release/0.4/configs/pcb) 中查看或下载。

In [4]:
! python -u tools/train.py -c ../faster_rcnn_r101_vd_fpn_1x.yml -o use_gpu=True \
      pretrain_weights=output/faster_rcnn_r101_vd_fpn_1x/best_model.pdmodel --eval
### ！！！CPU下训练执行如下指令！！！
#! python -u tools/train.py -c ../faster_rcnn_r34_1x_cpu.yml --eval -o use_gpu=False pretrain_weights=../faster_rcnn_r34_1x.pdparams

### 评估与预测
如果在训练中加了`--eval`参数，在模型训练完就可得到mAP指标，如果要对模型单独计算mAP，可以运行：

In [ ]:
 ! python -u tools/eval.py -c ../faster_rcnn_r101_vd_fpn_1x.yml \
                -o weights=output/faster_rcnn_r101_vd_fpn_1x/best_model
### ！！！CPU下执行如下指令！！！
# ! python -u tools/eval.py -c ../faster_rcnn_r34_1x_cpu.yml \
#                 -o weights=output/faster_rcnn_r34_1x_cpu/best_model \
#                 use_gpu=False

对印刷电路板（PCB）瑕疵数据集个别图片进行可视化预测，可以运行：

In [5]:
! python -u tools/infer.py -c ../faster_rcnn_r101_vd_fpn_1x.yml \
                --infer_img=../PCB_DATASET/images/04_missing_hole_13.jpg \
                -o weights=output/faster_rcnn_r101_vd_fpn_1x/best_model
### ！！！CPU下执行如下指令！！！
# ! python -u tools/infer.py -c ../faster_rcnn_r34_1x_cpu.yml \
#                 --infer_img=../PCB_DATASET/images/04_missing_hole_10.jpg \
#                 -o weights=output/faster_rcnn_r34_1x_cpu/best_model \
#                 use_gpu=False